In [3]:
!pip install --upgrade python-terrier
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

  Cloning https://github.com/Georgetown-IR-Lab/OpenNIR to /tmp/pip-req-build-o8fcz5dp
  Running command git clone -q https://github.com/Georgetown-IR-Lab/OpenNIR /tmp/pip-req-build-o8fcz5dp
  Cloning https://github.com/terrierteam/pyterrier_t5 to /tmp/pip-req-build-ytvxpq24
  Running command git clone -q https://github.com/terrierteam/pyterrier_t5 /tmp/pip-req-build-ytvxpq24


In [4]:
from tqdm import tqdm

import re
import pyterrier as pt

if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
import onir_pt
import pandas as pd
from pyterrier_t5 import MonoT5ReRanker
import pickle

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [5]:
from google.colab import drive
drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [6]:
query_variations =pd.read_csv('/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/data/variations_antique_labeled.csv')
query_variations["query"] = query_variations.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['original_query'].lower()), axis=1)
query_variations["variation"] = query_variations.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['variation'].lower()), axis=1)
query_variations["variation"] = query_variations.apply(lambda r: r['query'] if r['variation'].strip() == "" else r['variation'], axis=1)
query_variations['qid'] =query_variations['q_id'].astype(str)
query_variations.head(5)

,dataset,q_id,method,transformation_type,original_query,variation,valid,follow_category,query,qid
0,antique,1016750,WordSwapNeighboringCharacterSwap,mispelling,how do i erase all the contacts in my lg 6100 ...,how do i erase all the contacts in my gl 6100 ...,True,True,how do i erase all the contacts in my lg 6100 ...,1016750
1,antique,103298,WordSwapNeighboringCharacterSwap,mispelling,why precisely is foreign food considered ethnic,why precisely is foreign food considered ehtnic,True,True,why precisely is foreign food considered ethnic,103298
2,antique,1034050,WordSwapNeighboringCharacterSwap,mispelling,why does the earth keep getting hotter and hot...,why does the earth kepe getting hotter and hot...,True,True,why does the earth keep getting hotter and hot...,1034050
3,antique,1037897,WordSwapNeighboringCharacterSwap,mispelling,why doesn t salt lake city have its own sub ca...,why doesn t salt lkae city have its own sub ca...,True,True,why doesn t salt lake city have its own sub ca...,1037897
4,antique,1051346,WordSwapNeighboringCharacterSwap,mispelling,why is it when people cross the street they do...,why is it when people cross the strete they do...,True,True,why is it when people cross the street they do...,1051346


# dataset

In [11]:
def save_pickle(path,some_obj):
  with open(path, 'wb') as f:
    pickle.dump(some_obj, f)


def open_pickle(path):
  with open(path, 'rb') as handle:
      b = pickle.load(handle)
  return b 

## A1 : importing data

In [12]:
#dataset = pt.get_dataset("irds:antique/train/split200-valid")
#dataset
path_dataset= "/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/pickle/antique/"+"_dataset"
#save_pickle(path_dataset,dataset)
dataset =open_pickle(path_dataset)


In [13]:
#train
train_path  = "irds:antique/train"
train_ds =  pt.get_dataset(train_path)
#test 
test_path = "irds:antique/test"
test_ds =  pt.get_dataset(test_path)

test_topics = test_ds.get_topics()
train_topics =train_ds.get_topics()

print("#Q test ",len(test_topics))
print("#Q train ",len(train_topics))

#Q test  200
#Q train  2426


In [14]:
#Test  BM25
import os 
pt_index_path = '/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/index/antique/index_antique'

if not os.path.exists(pt_index_path + "/data.properties"):
  # création de l'index. Utilisation de l'itérateur pour parcourir la collection
  # on donne à l'index la fonction pour parcourir l'index avec l'itérateur  get_corpus_iter() 
  # On spécifie les champs à indexer et les meta-données à sauvegarder
  
  indexer = pt.index.IterDictIndexer(pt_index_path)
  indexref = indexer.index(dataset.get_corpus_iter(), fields=('docno', 'text'))
else:
  # dans le cas où l'index existe déjà
  indexref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(indexref)


antique/train/split200-valid documents:   0%|          | 0/403666 s<?, ?it/s]

## A2 : Test pour differents  

### sans test train

In [15]:
#table 5

cut_off = 100
metrics=["map", "recip_rank",'P_10', 'ndcg_cut.10']
baseline=0
def generate_original_scores(dataset,index,retrieval_model_name,metrics,cut_off=100,max_iter=400,train_ds=None):
      
      """
      fonction qui retourne le dataframe des scores des original queries

      **dataset : objet dataset 
      **train_ds : si modele neuronal  (objet data)
      **index : index de pt.Index
      **retrieval_model_name : string
      **metrics : liste de metriques
      **cut_off : param du retrieving

      """
      topics = dataset.get_topics()
      qrels = dataset.get_qrels()

      #modeles classiques
      if retrieval_model_name == "BM25":
        retrieval_model = pt.BatchRetrieve(index, wmodel=retrieval_model_name) % cut_off>> pt.text.get_text(dataset, 'text')
      elif retrieval_model_name == "RM3":
        #Relevance model 3 (RM3) is a traditional query expansion strategy based on language modeling. Several recent papers have shown that BM25 retrieval with RM3
        bm_25 = pt.BatchRetrieve(index, wmodel="BM25") 
        retrieval_model = bm_25 >>  pt.rewrite.RM3(index) >> bm_25

      #modeles neuronaux
      elif retrieval_model_name=="KNRM":
        knrm = onir_pt.reranker('knrm', 'wordvec_hash', config={'max_train_it':max_iter, 'pre_validate': False})
        retrieval_model = pt.BatchRetrieve(index, wmodel="BM25") % cut_off>> pt.text.get_text(train_ds, 'text') >> knrm
        #fit le modele neuronal 
        retrieval_model.fit(
            train_ds.get_topics(),
            train_ds.get_qrels(),
            dataset.get_topics(),
            dataset.get_qrels())
        #reranker = onir_pt.reranker()
        
        model_path = '/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/modeles/knrm'+retrieval_model_name+' '+str(max_iter)
        knrm.to_checkpoint(model_path)
      elif retrieval_model_name == "T5":
        reranker = MonoT5ReRanker()
        retrieval_model = pt.BatchRetrieve(index, wmodel="BM25") % cut_off >> pt.text.get_text(dataset, 'text') >> reranker
      elif retrieval_model_name=="BERT":
       bert = onir_pt.reranker('vanilla_transformer', 'bert', vocab_config={'train': True}, config={'max_train_it':max_iter, 'learning_rate': 1e-5, 'batch_size': 2,  'pre_validate': False, 'patience':50})
       retrieval_model = pt.BatchRetrieve(index, wmodel="BM25") % cut_off>> pt.text.get_text(train_ds, 'text') >> bert
       #fit le modele neuronal
       retrieval_model.fit(
           train_ds.get_topics(),
           train_ds.get_qrels(),
           dataset.get_topics(),
           dataset.get_qrels())
       #reranker = onir_pt.reranker()
       model_path = '/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/modeles/bert'+retrieval_model_name+' '+str(max_iter)
       bert.to_checkpoint(model_path)


      #epic or cknrm
      elif 'http' in retrieval_model_name:
        config = {}
        if 'epic' in retrieval_model_name:
          config = {'batch_size': 4}
        reranker = onir_pt.reranker.from_checkpoint(retrieval_model_name, config=config)
        retrieval_model = pt.BatchRetrieve(index, wmodel="BM25") % cut_off>> pt.text.get_text(train_ds, 'text') >> reranker
      

      """
      df = pt.Experiment(
          [retrieval_model],
          topics,
          qrels,

          eval_metrics= metrics
      )
      """
      df = None 
      return df,retrieval_model


#### Modeles usuels

In [18]:
#BM25
score_original_BM25,retrieval_model =generate_original_scores(dataset,index,"BM25",metrics)
score_original_BM25

[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-train.qrel
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-train.qrel: s] [626kB] [3.56MB/s]
[INFO] [starting] building docstore
docs_iter: 100%|██████████████████████| 403666/403666 [5.82s<0ms, 69363.03doc/s]
[INFO] [finished] docs_iter: [5.82s] [403666doc] [69345.42doc/s]
[INFO] [finished] building docstore [5.83s]


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(RankCutoff(BR(BM25), 100), pt.apply.??())",0.146804,0.459013,0.1595,0.232496


In [ ]:

#RM3
score_original_RM3,retrieval_model =generate_original_scores(dataset,index,"RM3",metrics)
score_original_RM3


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(BR(BM25), QueryExpansion(/cont...",0.146484,0.417824,0.154,0.218591


#### Modeles neuronnaux

In [ ]:
score_original_KNRM,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="KNRM",metrics=metrics,max_iter=100)
score_original_KNRM


[2022-04-15 15:13:24,976][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-04-15 15:13:36,511][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [11.53s]
[2022-04-15 15:15:07,492][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-15 15:15:07,728][onir_pt][DEBUG] [starting] training
[2022-04-15 15:15:07,729][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2022-04-15 15:15:10,019][onir_pt][DEBUG] not enough negs
[2022-04-15 15:15:13,947][onir_pt][DEBUG] not enough negs
[2022-04-15 15:15:14,280][onir_pt][DEBUG] not enough negs
[2022-04-15 15:15:18,012][onir_pt][DEBUG] not enough negs
[2022-04-15 15:15:18,520][onir_pt][DEBUG] [finished] train pairs: [10.79s] [1024it] [94.91it/s]
[2022-04-15 15:15:18,522][onir_pt][DEBUG] [finished] training [10.79s]
[2022-04-15 15:15:18,523][onir_pt][INFO] training   it=0 loss=0.2663
[2022-04-15 15:15:18,523][onir_pt][DEBUG] [starting] validation
[2022-04-15 15:15:18,524][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-15 15:15:18,525][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2022-04-15 15:15:27,433][onir_pt][DEBUG] [finished] batches: [8.91s] [4857it] [545.29it/s]
[2022-04-15 15:15:27,561][onir_pt][DEBUG] [finished] validation [9.04s]
[2022-04-15 15:15:27,563][onir_pt][INFO] validation it=0 map=0.0281 ndcg=0.1368 P_10=0.0365 <--
[2022-04-15 15:15:27,563][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-15 15:15:27,565][onir_pt][DEBUG] [starting] training
[2022-04-15 15:15:27,565][onir_pt][DEBUG] [starting] train pairs


...

batches:   0%|          | 0/4857 s<?, ?it/s]

[2022-04-15 15:42:18,963][onir_pt][DEBUG] [finished] batches: [15.21s] [4857it] [319.41it/s]
[2022-04-15 15:42:19,100][onir_pt][DEBUG] [finished] validation [15.35s]
[2022-04-15 15:42:19,101][onir_pt][INFO] validation it=83 map=0.1307 ndcg=0.2587 P_10=0.1565
[2022-04-15 15:42:19,102][onir_pt][INFO] early stopping; model reverting back to it=63
[2022-04-15 15:42:26,278][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-15 15:42:26,280][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2022-04-15 15:42:36,260][onir_pt][DEBUG] [finished] batches: [9.98s] [4857it] [486.70it/s]


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.13144,0.43339,0.153,0.217504


In [ ]:
score_original_KNRM


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.13144,0.43339,0.153,0.217504


#### CKRNM (link)

In [ ]:
score_original_CKNRM,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz",metrics=metrics,max_iter=100)
score_original_CKNRM

[2022-04-15 15:55:45,419][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz; consider adding expected_md5="6e57b8d728151debcb23479c8a41f4b0" to ensure data integrity.
[2022-04-15 15:55:45,424][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz s] [860k] [837kB/s]
[2022-04-15 15:55:45,459][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-04-15 15:55:51,944][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [6.49s]
[2022-04-15 15:56:04,993][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-15 15:56:05,251][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/2429 s<?, ?it/s]

[2022-04-15 15:58:16,745][onir_pt][DEBUG] [finished] batches: [02:11] [2429it] [18.47it/s]


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.115185,0.412362,0.138,0.20246


#### EPIC (link)

In [ ]:
score_original_EPIC,retrieval_model =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz",metrics=metrics,max_iter=100)
score_original_EPIC

[2022-04-15 16:01:04,403][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz; consider adding expected_md5="c184a924debf63b5115e34a3b92377e0" to ensure data integrity.
[2022-04-15 16:01:04,410][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz [23.22s] [495M] [29.6MB/s]


100%|██████████| 231508/231508 s<0ms, 319246.57B/s] 
100%|██████████| 433/433 s<0ms, 157718.94B/s]
100%|██████████| 440473133/440473133 [35.07s<0ms, 12558379.13B/s] 


[2022-04-15 16:03:59,693][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-15 16:03:59,933][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2022-04-15 16:12:38,147][onir_pt][DEBUG] [finished] batches: [08:38] [4857it] [ 9.37it/s]


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.160023,0.508168,0.183,0.266375


#### T5

In [ ]:
score_original_T5,retrieval_model =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="T5",metrics=metrics,max_iter=100)
score_original_T5


Downloading:   0%|          | 0.00/773k s<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k s<?, ?B/s]

Downloading:   0%|          | 0.00/1.80k s<?, ?B/s]

Downloading:   0%|          | 0.00/850M s<?, ?B/s]

monoT5:   0%|          | 0/4857 s<?, ?batches/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.205161,0.618335,0.216,0.335057


#### BERT

In [ ]:
score_original_BERT,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="BERT",metrics=metrics,max_iter=7)
score_original_BERT

[2022-04-20 09:04:33,840][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-20 09:04:34,088][onir_pt][DEBUG] [starting] training
[2022-04-20 09:04:34,090][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2022-04-20 09:04:48,468][onir_pt][DEBUG] not enough negs
[2022-04-20 09:05:16,787][onir_pt][DEBUG] not enough negs
[2022-04-20 09:05:21,147][onir_pt][DEBUG] not enough negs
[2022-04-20 09:06:10,766][onir_pt][DEBUG] not enough negs
[2022-04-20 09:06:17,668][onir_pt][DEBUG] [finished] train pairs: [01:44] [1024it] [ 9.89it/s]
[2022-04-20 09:06:17,669][onir_pt][DEBUG] [finished] training [01:44]
[2022-04-20 09:06:17,670][onir_pt][INFO] training   it=0 loss=0.2003
[2022-04-20 09:06:17,670][onir_pt][DEBUG] [starting] validation
[2022-04-20 09:06:17,670][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-20 09:06:17,685][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/9714 s<?, ?it/s]

[2022-04-20 09:09:30,183][onir_pt][DEBUG] [finished] batches: [03:12] [9714it] [50.46it/s]
[2022-04-20 09:09:30,298][onir_pt][DEBUG] [finished] validation [03:13]
[2022-04-20 09:09:32,037][onir_pt][INFO] validation it=0 map=0.1765 ndcg=0.3065 P_10=0.1950 <--
[2022-04-20 09:09:32,147][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-20 09:09:32,154][onir_pt][DEBUG] [starting] training
[2022-04-20 09:09:32,155][onir_pt][DEBUG] [starting] train pairs


...

batches:   0%|          | 0/9714 s<?, ?it/s]

[2022-04-20 09:39:11,957][onir_pt][DEBUG] [finished] batches: [03:12] [9714it] [50.48it/s]
[2022-04-20 09:39:12,071][onir_pt][DEBUG] [finished] validation [03:13]
[2022-04-20 09:39:13,822][onir_pt][INFO] validation it=6 map=0.2273 ndcg=0.3539 P_10=0.2325 <--
[2022-04-20 09:39:50,634][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-20 09:39:50,640][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/9714 s<?, ?it/s]

[2022-04-20 09:43:03,339][onir_pt][DEBUG] [finished] batches: [03:13] [9714it] [50.41it/s]


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.227294,0.66676,0.2325,0.354889


### Variation methods

In [16]:

def get_variation_score(dataset,retrieval_model_name,retrieval_model,query_variations):
  variation_methods = []
  res_per_variation = []
  print()
  for method in query_variations['method'].unique():
    query_variation = query_variations[query_variations['method'] == method]
    print(method)
    query_variation['query'] = query_variation['variation']
    res_per_variation.append(retrieval_model.transform(query_variation[['qid', 'query']]))
    variation_methods.append("{}+QueriesFrom{}".format(retrieval_model_name, method))

  df = pt.Experiment(
        [retrieval_model] + res_per_variation,
        #requetes
        query_variations[['query','qid']].drop_duplicates(),
        #jugement de pertinence
        dataset.get_qrels(),
        metrics,
        baseline=0,
        names = [retrieval_model_name]+variation_methods
        )
  path = "/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/pickle/antique/variations"+"_"+retrieval_model_name+"_"+"antique"
  save_pickle(path,df)
  df.head(10)
    
  return df





### Modeles usuels

In [31]:
#BM25

#scores_original_BM25,retrieval_model =generate_original_scores(dataset,index,"BM25",metrics)
scores_variation_BM25 = get_variation_score(dataset,"BM25",retrieval_model,query_variations)


WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


WordSwapQWERTY
WordSwapRandomCharacterSubstitution
naturality_by_removing_stop_words
summarization_with_t5-base_from_description_to_title
WordInnerSwapRandom
back_translation_pivot_language_de
ramsrigouthamg/t5_paraphraser
WordSwapEmbedding
WordSwapWordNet


In [32]:
scores_variation_BM25.head(10)


,name,map,recip_rank,P_10,ndcg_cut.10,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P_10 +,P_10 -,P_10 p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,BM25,0.146804,0.459013,0.1595,0.232496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25+QueriesFromWordSwapNeighboringCharacterSwap,0.100830,0.341528,0.1085,0.161718,38.0,108.0,4.286676e-08,32.0,73.0,1.756439e-06,9.0,59.0,9.392530e-09,26.0,81.0,2.761121e-09
2,BM25+QueriesFromWordSwapQWERTY,0.102497,0.319541,0.1130,0.164768,35.0,101.0,4.761817e-08,22.0,77.0,8.284207e-09,10.0,56.0,1.799167e-08,20.0,79.0,1.299733e-09
3,BM25+QueriesFromWordSwapRandomCharacterSubstit...,0.105295,0.339668,0.1130,0.166315,34.0,107.0,2.319732e-07,25.0,76.0,3.041169e-07,9.0,65.0,4.240100e-09,22.0,81.0,8.762220e-09
4,BM25+QueriesFromnaturality_by_removing_stop_words,0.146054,0.461943,0.1585,0.231195,18.0,14.0,3.687120e-01,13.0,5.0,4.060475e-01,2.0,5.0,5.284356e-01,11.0,8.0,4.690199e-01
5,BM25+QueriesFromsummarization_with_t5-base_fro...,0.114148,0.366476,0.1160,0.171695,40.0,98.0,2.667069e-06,29.0,62.0,2.755037e-05,12.0,63.0,4.640107e-07,23.0,78.0,3.074479e-07
6,BM25+QueriesFromWordInnerSwapRandom,0.146850,0.459013,0.1595,0.232726,1.0,0.0,3.185249e-01,0.0,0.0,NaN,0.0,0.0,NaN,1.0,0.0,3.185249e-01
7,BM25+QueriesFromback_translation_pivot_languag...,0.102435,0.335228,0.1120,0.162992,29.0,85.0,7.163481e-07,23.0,67.0,1.598084e-06,12.0,56.0,4.537875e-07,18.0,65.0,4.033577e-08
8,BM25+QueriesFromramsrigouthamg/t5_paraphraser,0.142428,0.453519,0.1535,0.225769,26.0,32.0,2.161253e-01,19.0,17.0,5.935664e-01,5.0,14.0,5.758621e-02,15.0,21.0,1.777499e-01
9,BM25+QueriesFromWordSwapEmbedding,0.112839,0.374335,0.1170,0.178025,29.0,95.0,5.005837e-06,20.0,63.0,5.868535e-05,7.0,54.0,9.869501e-08,21.0,66.0,4.962337e-07


In [37]:
#RM3
score_original_RM3,retrieval_model =generate_original_scores(dataset,index,"RM3",metrics)
scores_variation_RM3=get_variation_score(dataset,"RM3",retrieval_model,query_variations)




WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


WordSwapQWERTY
WordSwapRandomCharacterSubstitution
naturality_by_removing_stop_words
summarization_with_t5-base_from_description_to_title
WordInnerSwapRandom
back_translation_pivot_language_de
ramsrigouthamg/t5_paraphraser
WordSwapEmbedding
WordSwapWordNet


### Modeles neuronaux

In [38]:
#CKNRM
score_original_CKNRM,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz",metrics=metrics,max_iter=100)
scores_variation_CKNRM=get_variation_score(dataset,"CKNRM",retrieval_model,query_variations)


[2022-04-30 11:22:39,289][onir_pt][INFO] using cached checkpoint: /root/data/onir/model_checkpoints/52177bd86fe00fbf6a41fbb843b981b1
[2022-04-30 11:22:39,298][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-04-30 11:22:50,942][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [11.64s]

WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


[2022-04-30 11:23:00,554][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:23:00,878][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/2339 s<?, ?it/s]

[2022-04-30 11:24:03,772][onir_pt][DEBUG] [finished] batches: [01:03] [2339it] [37.19it/s]
WordSwapQWERTY
[2022-04-30 11:24:08,852][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:24:08,853][onir_pt][DEBUG] [starting] batches


...

[2022-04-30 11:33:27,778][onir_pt][DEBUG] [finished] batches: [01:06] [2407it] [36.38it/s]
WordSwapWordNet
[2022-04-30 11:33:33,260][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:33:33,262][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/2501 s<?, ?it/s]

[2022-04-30 11:34:40,642][onir_pt][DEBUG] [finished] batches: [01:07] [2501it] [37.12it/s]
[2022-04-30 11:34:46,216][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:34:46,218][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/2429 s<?, ?it/s]

[2022-04-30 11:35:51,547][onir_pt][DEBUG] [finished] batches: [01:05] [2429it] [37.18it/s]


In [39]:
#EPIC
score_original_EPIC,retrieval_model =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz",metrics=metrics,max_iter=100)
scores_variation_EPIC=get_variation_score(dataset,"EPIC",retrieval_model,query_variations)

[2022-04-30 11:35:52,508][onir_pt][INFO] using cached checkpoint: /root/data/onir/model_checkpoints/f95b2957293fb574a85f71daeb9523d1

WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


[2022-04-30 11:37:46,965][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:37:47,219][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4678 s<?, ?it/s]

[2022-04-30 11:41:51,059][onir_pt][DEBUG] [finished] batches: [04:04] [4678it] [19.18it/s]
WordSwapQWERTY
[2022-04-30 11:41:56,102][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:41:56,107][onir_pt][DEBUG] [starting] batches


...

[2022-04-30 12:18:57,484][onir_pt][DEBUG] [finished] batches: [04:38] [4814it] [17.29it/s]
WordSwapWordNet
[2022-04-30 12:19:02,840][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 12:19:02,845][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/5001 s<?, ?it/s]

[2022-04-30 12:23:43,548][onir_pt][DEBUG] [finished] batches: [04:41] [5001it] [17.82it/s]
[2022-04-30 12:23:49,302][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 12:23:49,307][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2022-04-30 12:28:27,418][onir_pt][DEBUG] [finished] batches: [04:38] [4857it] [17.46it/s]


In [40]:
#T5
score_original_T5,retrieval_model =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="T5",metrics=metrics,max_iter=100)
scores_variation_T5=get_variation_score(dataset,"T5",retrieval_model,query_variations)


WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


monoT5:   0%|          | 0/4678 s<?, ?batches/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (790 > 512). Running this sequence through the model will result in indexing errors


WordSwapQWERTY


...

WordSwapEmbedding


monoT5:   0%|          | 0/4814 s<?, ?batches/s]

WordSwapWordNet


monoT5:   0%|          | 0/5001 s<?, ?batches/s]

monoT5:   0%|          | 0/4857 s<?, ?batches/s]

In [17]:
#KNRM
score_original_KNRM,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="KNRM",metrics=metrics,max_iter=10)
scores_variation_KNRM=get_variation_score(dataset,"KNRM",retrieval_model,query_variations)

config file not found: config
[2022-04-30 13:24:45,642][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-04-30 13:24:56,424][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [10.78s]
[2022-04-30 13:26:19,074][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:26:32,899][onir_pt][DEBUG] [starting] training
[2022-04-30 13:26:32,901][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2022-04-30 13:26:34,944][onir_pt][DEBUG] not enough negs
[2022-04-30 13:26:36,598][onir_pt][DEBUG] not enough negs
[2022-04-30 13:26:36,851][onir_pt][DEBUG] not enough negs
[2022-04-30 13:26:39,775][onir_pt][DEBUG] not enough negs
[2022-04-30 13:26:40,165][onir_pt][DEBUG] [finished] train pairs: [7.26s] [1024it] [140.99it/s]
[2022-04-30 13:26:40,167][onir_pt][DEBUG] [finished] training [7.27s]
[2022-04-30 13:26:40,167][onir_pt][INFO] training   it=0 loss=0.2663
[2022-04-30 13:26:40,168][onir_pt][DEBUG] [starting] validation
[2022-04-30 13:26:40,168][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:26:40,171][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2022-04-30 13:26:46,806][onir_pt][DEBUG] [finished] batches: [6.63s] [4857it] [732.10it/s]
[2022-04-30 13:26:46,918][onir_pt][DEBUG] [finished] validation [6.75s]
[2022-04-30 13:26:46,931][onir_pt][INFO] validation it=0 map=0.0281 ndcg=0.1368 P_10=0.0365 <--
[2022-04-30 13:26:46,932][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:26:46,933][onir_pt][DEBUG] [starting] training
[2022-04-30 13:26:46,934][onir_pt][DEBUG] [starting] train pairs


...

[2022-04-30 13:30:46,926][onir_pt][DEBUG] [finished] batches: [7.20s] [4814it] [668.41it/s]
WordSwapWordNet
[2022-04-30 13:30:52,545][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:30:52,547][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/5001 s<?, ?it/s]

[2022-04-30 13:30:59,803][onir_pt][DEBUG] [finished] batches: [7.26s] [5001it] [689.29it/s]
[2022-04-30 13:31:05,735][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:31:05,736][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2022-04-30 13:31:12,889][onir_pt][DEBUG] [finished] batches: [7.15s] [4857it] [679.12it/s]


In [18]:
#BERT
score_original_BERT,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="BERT",metrics=metrics,max_iter=3)
scores_variation_BERT=get_variation_score(dataset,"BERT",retrieval_model,query_variations)

[2022-04-30 13:32:36,939][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:32:37,371][onir_pt][DEBUG] [starting] training
[2022-04-30 13:32:37,373][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2022-04-30 13:32:52,584][onir_pt][DEBUG] not enough negs
[2022-04-30 13:33:22,364][onir_pt][DEBUG] not enough negs
[2022-04-30 13:33:27,026][onir_pt][DEBUG] not enough negs
[2022-04-30 13:34:20,057][onir_pt][DEBUG] not enough negs
[2022-04-30 13:34:27,525][onir_pt][DEBUG] [finished] train pairs: [01:50] [1024it] [ 9.30it/s]
[2022-04-30 13:34:27,527][onir_pt][DEBUG] [finished] training [01:50]
[2022-04-30 13:34:27,529][onir_pt][INFO] training   it=0 loss=0.2002
[2022-04-30 13:34:27,532][onir_pt][DEBUG] [starting] validation
[2022-04-30 13:34:27,533][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:34:27,543][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/9714 s<?, ?it/s]

[2022-04-30 13:38:02,814][onir_pt][DEBUG] [finished] batches: [03:35] [9714it] [45.12it/s]
[2022-04-30 13:38:02,927][onir_pt][DEBUG] [finished] validation [03:35]
[2022-04-30 13:38:05,180][onir_pt][INFO] validation it=0 map=0.1819 ndcg=0.3127 P_10=0.1985 <--
[2022-04-30 13:38:05,182][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:38:05,188][onir_pt][DEBUG] [starting] training
[2022-04-30 13:38:05,188][onir_pt][DEBUG] [starting] train pairs


...

[2022-04-30 14:21:02,873][onir_pt][DEBUG] [finished] batches: [03:30] [9628it] [45.91it/s]
WordSwapWordNet
[2022-04-30 14:21:08,227][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 14:21:08,233][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/10001 s<?, ?it/s]

[2022-04-30 14:24:45,033][onir_pt][DEBUG] [finished] batches: [03:37] [10001it] [46.13it/s]
[2022-04-30 14:24:50,898][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 14:24:50,907][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/9714 s<?, ?it/s]

[2022-04-30 14:28:24,746][onir_pt][DEBUG] [finished] batches: [03:34] [9714it] [45.43it/s]


In [19]:
scores_variation_BERT

,name,map,recip_rank,P_10,ndcg_cut.10,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P_10 +,P_10 -,P_10 p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,BERT,0.212405,0.661425,0.2250,0.337762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BERT+QueriesFromWordSwapNeighboringCharacterSwap,0.149420,0.492970,0.1580,0.240649,35.0,104.0,7.351564e-10,18.0,65.0,1.514073e-08,10.0,71.0,3.731762e-12,33.0,98.0,2.097266e-11
2,BERT+QueriesFromWordSwapQWERTY,0.153641,0.470606,0.1630,0.246509,38.0,100.0,6.605792e-10,21.0,72.0,2.097112e-10,14.0,70.0,2.126970e-10,30.0,99.0,3.716093e-11
3,BERT+QueriesFromWordSwapRandomCharacterSubstit...,0.147815,0.462798,0.1575,0.235492,30.0,117.0,4.421127e-12,16.0,75.0,1.195259e-11,9.0,74.0,1.133041e-12,26.0,108.0,6.157993e-14
4,BERT+QueriesFromnaturality_by_removing_stop_words,0.192243,0.593390,0.2025,0.304311,31.0,101.0,2.252081e-06,16.0,49.0,2.392379e-04,8.0,42.0,5.230921e-07,35.0,83.0,3.690131e-07
5,BERT+QueriesFromsummarization_with_t5-base_fro...,0.134953,0.451254,0.1450,0.216391,16.0,131.0,2.721310e-16,14.0,82.0,5.490244e-14,6.0,93.0,8.395444e-18,15.0,120.0,2.756083e-20
6,BERT+QueriesFromWordInnerSwapRandom,0.207107,0.638595,0.2180,0.328411,39.0,68.0,1.326605e-01,13.0,32.0,1.053707e-01,6.0,19.0,2.648831e-02,33.0,67.0,2.665565e-02
7,BERT+QueriesFromback_translation_pivot_languag...,0.154037,0.483510,0.1690,0.252219,34.0,95.0,6.262400e-09,13.0,64.0,3.093437e-10,11.0,57.0,4.457653e-08,28.0,88.0,4.092189e-10
8,BERT+QueriesFromramsrigouthamg/t5_paraphraser,0.198364,0.607017,0.2130,0.316993,34.0,50.0,4.142247e-03,14.0,28.0,1.204934e-03,5.0,24.0,7.708280e-04,26.0,44.0,7.390143e-04
9,BERT+QueriesFromWordSwapEmbedding,0.158127,0.500002,0.1735,0.259146,33.0,102.0,5.068912e-10,18.0,59.0,3.587977e-09,11.0,66.0,4.773459e-10,32.0,94.0,3.656861e-11
